In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Helper Functions

In [75]:
#init Weight
def init_weight(shape,name_W):
    init_rand_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_rand_dist,name=name_W)

In [76]:
#init Bias
def init_bias(shape,name_b):
    init_bias_vals = tf.constant(value=0.1,shape=shape)
    return tf.Variable(initial_value=init_bias_vals,name=name_b)

In [98]:
#Conv2D

def conv2d(X,W,name_conv):
    # X --> [batch,H,W,Channels]
    # W --> [filter H , filter W , Channel In , Channel Out]
    
    
    
    return tf.nn.conv2d(X,W,strides=[1,1,1,1],padding="SAME",name=name_conv)
    
    

In [99]:
#Pooling



def max_pooling_2by2(X):
    # X --> [batch,H,W,Channels]
    
    return tf.nn.max_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    

In [100]:
#Convolutional Layes

def convolutional_layer(input_x, shape,name_W,name_b,name_conv):
    W = init_weight(shape = shape,name_W=name_W)
    b = init_bias(shape = [shape[3]],name_b = name_b)
    
    
    return tf.nn.relu(conv2d(input_x,W, name_conv =name_conv)+b )




In [117]:
#Normal Layer (fully connected Layer)

def normal_full_layer(input_layer,size,name_W,name_b):
    
    input_size = int(input_layer.get_shape()[1])
    W = init_weight([input_size,size],name_W=name_W)
    b = init_bias([size],name_b=name_b)
    
    return tf.matmul(input_layer , W) + b

# Building Model 

In [102]:
X = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [103]:
X_image = tf.reshape(X,[-1,28,28,1])

# Model Layers



In [109]:
convo_1 = convolutional_layer( X_image , shape = [5,5,1,32] , name_W = "W_conv1" , name_b = "bias_Conv1" , name_conv = "Conv_1")
convo_1_pooling = max_pooling_2by2(convo_1)

In [111]:
convo_2 = convolutional_layer( convo_1_pooling , shape = [5,5,32,64] , name_W = "W_conv2" , name_b = "bias_Conv2" , name_conv = "Conv_2")
convo_2_pooling = max_pooling_2by2(convo_1)

In [119]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024,name_W="full_layer_W",name_b="full_layer_b"))

 # Dropout

In [121]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropot = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [123]:
y_pred = normal_full_layer(full_one_dropot,10,name_W = 'out_W',name_b='out_b' )

# Loss Function

In [124]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true , logits= y_pred ))

# optimizer

In [126]:
optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

# Session Running

In [127]:
init = tf.global_variables_initializer()

In [ ]:
steps = 5000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range (steps) :
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')
        